In [1]:
!pip install gpytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 6.4 MB/s eta 0:00:00


In [2]:
from google.colab import files

In [3]:
!pip install cocopp
!git clone https://github.com/numbbo/coco.git
!cd coco; python do.py run-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 4.9 MB/s eta 0:00:00
Cloning into 'coco'...
remote: Enumerating objects: 57651, done.
remote: Counting objects: 100% (3506/3506), done.
remote: Compressing objects: 100% (1246/1246), done.
remote: Total 57651 (delta 2388), reused 3120 (delta 2162), pack-reused 54145
Receiving objects: 100% (57651/57651), 277.59 MiB | 24.43 MiB/s, done.
Resolving deltas: 100% (37264/37264), done.
AML	['code-experiments/src/coco_random.c', 'code-experiments/src/coco_suite.c', 'code-experiments/src/coco_observer.c', 'code-experiments/src/coco_archive.c', 'code-experiments/src/coco_runtime_c.c'] -> code-experiments/build/python/cython/coco.c
EXPAND	code-experiments/build/python/cython/coco.c.in to code-experiments/build/python/cython/coco.c
EXPAND	code-experiments/src/coco.h to code-experiments/build/python/cython/coco.h
COPY	code-experiments/src/bbob2009_testcases.txt -> code-experiments/build/python/bbob2009_testcases.txt
COPY	code-experiments/src/

In [4]:
!cd coco/code-experiments/build/python; git clone https://github.com/uber-research/TuRBO

Cloning into 'TuRBO'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 25 (delta 4), reused 2 (delta 2), pack-reused 12
Receiving objects: 100% (25/25), 70.05 KiB | 771.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.


In [8]:
%%writefile simplest_run.py

#!/usr/bin/env python
"""A short and simple example experiment with restarts.

The code is fully functional but mainly emphasises on readability.
Hence produces only rudimentary progress messages and does not provide
batch distribution or timing prints, as `example_experiment2.py` does.

To apply the code to a different solver, `fmin` must be re-assigned or
re-defined accordingly. For example, using `cma.fmin` instead of
`scipy.optimize.fmin` can be done like::

>>> import cma  # doctest:+SKIP
>>> def fmin(fun, x0):
...     return cma.fmin(fun, x0, 2, {'verbose':-9})

"""
from __future__ import division, print_function
import cocoex, cocopp  # experimentation and post-processing modules
import scipy.optimize  # to define the solver to be benchmarked
from TuRBO.turbo import Turbo1
import numpy as np
from numpy.random import rand  # for randomised restarts
import os, webbrowser  # to show post-processed results in the browser

def fmin(problem, x0, verbose):
    dim = len(x0)
    fmin_ = Turbo1(problem,
                    lb = -5 * np.ones(dim),
                    ub = 10 * np.ones(dim),
                    n_init = 20,
                    max_evals = 100,
                    verbose = False,
                    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
                    max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
                    n_training_steps=50,  # Number of steps of ADAM to learn the hypers
                    min_cuda=1024,  # Run on the CPU for small datasets
                    device="cpu",  # "cpu" or "cuda"
                    dtype="float64")
    return fmin_.optimize()

### input
suite_name = "bbob"
output_folder = "scipy-optimize-fmin"
# fmin = Turbo1
budget_multiplier = 1  # increase to 10, 100, ...

### prepare
suite = cocoex.Suite(suite_name, "", "")
observer = cocoex.Observer(suite_name, "result_folder: " + output_folder)
minimal_print = cocoex.utilities.MiniPrint()

### go
for problem in suite:  # this loop will take several minutes or longer
    problem.observe_with(observer)  # generates the data for cocopp post-processing
    x0 = problem.initial_solution
    # apply restarts while neither the problem is solved nor the budget is exhausted
    while (problem.evaluations < problem.dimension * budget_multiplier
           and not problem.final_target_hit):
        fmin(problem, x0, verbose=False)  # here we assume that `fmin` evaluates the final/returned solution
        x0 = problem.lower_bounds + ((rand(problem.dimension) + rand(problem.dimension)) *
                    (problem.upper_bounds - problem.lower_bounds) / 2)
    minimal_print(problem, final=problem.index == len(suite) - 1)

### post-process data
cocopp.main(observer.result_folder)  # re-run folders look like "...-001" etc
webbrowser.open("file://" + os.getcwd() + "/ppdata/index.html")

Writing simplest_run.py


In [ ]:
!ls; mv simplest_run.py coco/code-experiments/build/python;

In [11]:
!cd coco/code-experiments/build/python; python simplest_run.py

COCO INFO: Results will be output to folder exdata/scipy-optimize-fmin-001
2D 11h13:55s
.Traceback (most recent call last):
  File "/content/coco/code-experiments/build/python/simplest_run.py", line 60, in <module>
    fmin(problem, x0, verbose=False)  # here we assume that `fmin` evaluates the final/returned solution
  File "/content/coco/code-experiments/build/python/simplest_run.py", line 40, in fmin
    return fmin_.optimize()
  File "/content/coco/code-experiments/build/python/TuRBO/turbo/turbo_1.py", line 275, in optimize
    X_cand, y_cand, _ = self._create_candidates(
  File "/content/coco/code-experiments/build/python/TuRBO/turbo/turbo_1.py", line 173, in _create_candidates
    gp = train_gp(
  File "/content/coco/code-experiments/build/python/TuRBO/turbo/gp.py", line 90, in train_gp
    loss = -mll(output, train_y)
  File "/usr/local/lib/python3.10/dist-packages/gpytorch/module.py", line 31, in __call__
    outputs = self.forward(*inputs, **kwargs)
  File "/usr/local/lib/pyth